In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load the downloaded Yahoo text file and build df (remove Dividend lines)
with open('ISF.L.yahoo.txt', 'r') as f:
    lines = f.readlines()

filtered_lines = [line for line in lines if 'Dividend' not in line]
df = pd.read_csv(pd.io.common.StringIO(''.join(filtered_lines)), sep='\t')

# Clean columns and dates
df.columns = df.columns.str.strip()
df.rename(columns={'Adj Close': 'AdjClose'}, inplace=True)
df['Date'] = df['Date'].str.replace('Sept', 'Sep')
df['Date'] = pd.to_datetime(df['Date'], format='%d %b %Y')
df = df.set_index('Date').sort_index()
df.head()

In [ ]:
# Quick plot of AdjClose to verify data
df['AdjClose'].plot(title='ISF.L AdjClose')

## Backtest engine and strategies

The next code cell defines a BacktestEngine that runs strategies which are called each step with (cash, equity_value, price, step_index).
Each strategy returns an amount (positive means buy equity with that amount of cash, negative means sell that amount of equity to cash).
Trades are executed at the next day's price. The engine starts with 1000 cash by default.

In [ ]:
class BacktestEngine:
    def __init__(self, price_series, initial_cash=1000.0):
        # price_series: pd.Series indexed by date
        self.prices = price_series.dropna().copy()
        self.initial_cash = float(initial_cash)

    def run(self, strategy):
        # strategy must implement step(cash, equity_value, price, step_index) -> amount (to be executed next day)
        dates = self.prices.index.tolist()
        n = len(self.prices)

        cash = self.initial_cash
        shares = 0.0
        pending_amount = 0.0  # amount scheduled at previous step to execute at current price

        rows = []
        for i, date in enumerate(dates):
            price = float(self.prices.iloc[i])

            # equity value at current price
            equity_value = shares * price
            total = cash + equity_value

            # Execute the pending trade (which was scheduled at i-1) at today's price
            executed_amount = 0.0
            executed_price = None
            if i > 0 and pending_amount != 0.0:
                # buying (positive) or selling (negative) executed using today's price
                executed_amount = pending_amount
                executed_price = price
                # adjust shares and cash
                shares += executed_amount / price
                cash -= executed_amount
                # numerical guard
                shares = float(shares)
                cash = float(cash)
                equity_value = shares * price
                total = cash + equity_value

            # Ask the strategy what to schedule for NEXT day
            # Provide current cash and equity_value (at current day's price)
            try:
                scheduled = float(strategy.step(cash, equity_value, price, i))
            except AttributeError:
                # allow passing a plain function too: func(cash, equity, price, i)
                scheduled = float(strategy(cash, equity_value, price, i))

            # If scheduling a buy (positive), ensure we don't schedule more than current cash
            if scheduled > 0 and scheduled > cash:
                scheduled = float(cash)
            # If scheduling a sell (negative), ensure we don't sell more than equity value
            if scheduled < 0 and abs(scheduled) > equity_value:
                scheduled = -float(equity_value)

            # record row for this date (state after executing pending and scheduling next)
            rows.append({
                'date': date,
                'price': price,
                'cash': cash,
                'shares': shares,
                'equity_value': equity_value,
                'total': total,
                'executed_amount': executed_amount,
                'executed_price': executed_price,
                'scheduled_for_next': scheduled,
            })

            # pending becomes the scheduled amount for next iteration (which will execute at i+1 price)
            pending_amount = scheduled

        # final DataFrame
        res = pd.DataFrame(rows).set_index('date')
        return res

### Sample strategies

Two simple strategy implementations: AllInStrategy (put all cash into equity at first opportunity), and RebalanceEveryNStrategy (every N steps rebalance to a target equity ratio).

In [ ]:
class AllInStrategy:
    def __init__(self):
        self.invested = False

    def step(self, cash, equity_value, price, i):
        # If we still have cash, schedule to invest all cash next day once
        if (not self.invested) and cash > 0:
            self.invested = True
            return float(cash)
        return 0.0

class RebalanceEveryNStrategy:
    def __init__(self, n=5, target_equity_ratio=0.7):
        self.n = int(n)
        self.target_equity_ratio = float(target_equity_ratio)

    def step(self, cash, equity_value, price, i):
        # Every n steps (including step 0) schedule a rebalance at next day price
        if (i % self.n) == 0:
            total = cash + equity_value
            if total <= 0:
                return 0.0
            desired_equity = total * self.target_equity_ratio
            # amount to move into equity (positive) or to cash (negative)
            diff = desired_equity - equity_value
            # cap buys to available cash, cap sells to available equity
            if diff > 0:
                return float(min(diff, cash))
            else:
                return float(max(diff, -equity_value))
        return 0.0


### Plotting utilities

The following cell contains simple plotting helpers to compare NAVs and allocations across strategies.

In [ ]:
def plot_nav(results_dict, title='NAV comparison'):
    plt.figure(figsize=(10, 6))
    for name, df_res in results_dict.items():
        plt.plot(df_res.index, df_res['total'], label=name)
    plt.legend()
    plt.title(title)
    plt.ylabel('Total NAV')
    plt.xlabel('Date')
    plt.grid(True)
    plt.show()

def plot_allocations(df_res, title=None):
    # stacked area for cash vs equity
    alloc = pd.DataFrame({'cash': df_res['cash'], 'equity': df_res['equity_value']}, index=df_res.index)
    alloc_frac = alloc.div(alloc.sum(axis=1), axis=0)
    alloc_frac.plot.area(figsize=(10, 4), alpha=0.6)
    plt.title(title or 'Allocation (fraction)')
    plt.ylabel('Fraction')
    plt.xlabel('Date')
    plt.show()


### Run demo backtests

Run the AllInStrategy and RebalanceEveryNStrategy and compare results. The engine starts with 1000 cash.

In [ ]:
# Prepare price series (ensure name is present)
price_series = df['AdjClose']

# Instantiate engine and strategies
engine = BacktestEngine(price_series, initial_cash=1000.0)
s_allin = AllInStrategy()
s_reb5 = RebalanceEveryNStrategy(n=5, target_equity_ratio=0.7)

# Run backtests
res_allin = engine.run(s_allin)
res_reb5 = engine.run(s_reb5)

# Add equity fraction column for plotting convenience
res_allin['equity_frac'] = res_allin['equity_value'] / res_allin['total']
res_reb5['equity_frac'] = res_reb5['equity_value'] / res_reb5['total']

# Show tail of results
print('All-in strategy (tail):')
print(res_allin.tail())
print('\nRebalance every 5 steps (tail):')
print(res_reb5.tail())

# Plot NAV comparison
plot_nav({'AllIn': res_allin, 'Rebalance5': res_reb5}, title='Strategy NAVs')

# Plot allocation fractions for rebalance strategy
plot_allocations(res_reb5, title='Rebalance5 allocations (fraction)')
